In [1]:
!git clone https://github.com/ozzux/CDD

Cloning into 'DICA'...
remote: Enumerating objects: 34577, done.
remote: Counting objects: 100% (11301/11301), done.
remote: Compressing objects: 100% (10166/10166), done.
Receiving objects: 100% (34577/34577), 1.16 GiB | 19.11 MiB/s, done.
remote: Total 34577 (delta 168), reused 11290 (delta 157), pack-reused 23276
Resolving deltas: 100% (215/215), done.


In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.5/792.5 kB 10.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [59]:
from ultralytics import YOLO
import cv2
import torch

def seg(img_path, model_path):
    model = YOLO(model_path)

    results = model.predict(source=img_path)

    result = results[0]

    # get array results
    masks = result.masks.data
    boxes = result.boxes.data
    # extract classes
    clss = boxes[:, 5]
    # get indices of results where class is 0 (people in COCO)
    people_indices = torch.where(clss == 0)
    # use these indices to extract the relevant masks
    people_masks = masks[people_indices]
    # scale for visualizing results
    people_mask = torch.any(people_masks, dim=0).int() * 255
    return people_mask.cpu().numpy()

Upload the segmentationmodel.pt from the zipped model

In [74]:
IMAGE_PATH = "/content/DICA/convdata/ADEM/ADEM_2.jpg" # INPUT IMAGE PATH HERE

In [75]:
result = seg(IMAGE_PATH,'/content/segmentationmodel.pt')
cv2.imwrite('temp.png',result)


image 1/1 /content/DICA/convdata/ADEM/ADEM_2.jpg: 640x576 11 lesions, 5888.3ms
Speed: 4.9ms preprocess, 5888.3ms inference, 37.3ms postprocess per image at shape (1, 3, 640, 576)


True

In [76]:
image = cv2.imread('/content/temp.png')

x = []
x.append(image/255)
x = np.array(x)

In [77]:
import tensorflow as tf
import tensorflow_hub as hub

model = tf.keras.models.load_model('/content/DICA/convnet.model')

In [78]:
ans = model.predict(x)
print(ans[0][0])
ans = round(ans[0][0])

if ans == 0:
  print("Likely MS")
elif ans > 0.45:
  if ans < 0.55:
    print('Unclear')
  else:
    print('Likely ADEM')

1/1 [==============================] - 1s 1s/step
0.629177
Likely ADEM
